In [18]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

In [19]:
import os, sys, time

In [20]:
import ROOT

In [21]:
#Whoops, had to do scram b in src directory to build all the libraries and such... can't find the postproc otherwise.
#reload kernel as well if above done after failing to import postproc
from importlib import import_module
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor

In [81]:
from PhysicsTools.NanoAODTools.postprocessing.framework.datamodel import Collection, Object
from PhysicsTools.NanoAODTools.postprocessing.framework.eventloop import Module

In [158]:
class testing(Module):
    def __init__(self):
        self.counting = 0
        pass
    def beginJob(self):
        pass
    def endJob(self):
        pass
    #def beginFile(self, inputFile, outputFile):
    #def beginFile(self):
        #pass
    #def endFile(self, inputFile, outputFile):
    #def endFile(self):
        #pass
    def analyze(self, event):
        """process event, return True (go to next module) or False (fail, go to next event)"""
        self.counting += 1
        if self.counting > 7:
            return False
        electrons = Collection(event, "Electron")
        photons = Collection(event, "Photon")
        muons = Collection(event, "Muon")
        taus = Collection(event, "Tau")
        jets = Collection(event, "Jet")
        met = Object(event, "MET")
        #met = event.MET
        run = event.run
        theEvent = event.event
        #te = Object(event, "event") #returns a unicode type?
        triggers = Object(event, "HLT") #this works...
        #flags = Flag(event, "Flag") # does not work
        PVertex = [event.PV_x, event.PV_y, event.PV_z]
        PV = Object(event, "PV")
        SVertices = Collection(event, "SV")
        genParts = Collection(event, "GenPart")
        genJets = Collection(event, "GenJet")
        genFatJets = Collection(event, "GenJetAK8")
        eventSum = ROOT.TLorentzVector()
        print("Run: {0:>8d} Event: {1:>8d}".format(run, theEvent))
        print("PV  X: {0: >5.3f} Y: {1: >5.3f} Z: {2:5.3f}".format(PVertex[0], PVertex[1], PVertex[2]))
        print("PV  X: {0: >5.3f} Y: {1: >5.3f} Z: {2:5.3f} nDoF: {3: >5f} Chi^2: {4: >5.3f}".format(
            PV.x,PV.y, PV.z, PV.ndof, PV.chi2))
        if len(SVertices) > 0:   
            print("nSV: {0: >3d} SV[0] Decay Length:{1: >5.3f}".format(event.nSV, SVertices[0].dlen ))
        else:
            print("nSV: {0: >3d}".format(event.nSV))
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Muon", "Pt", "Eta", "Phi"))
        for nm, lep in enumerate(muons) :
            eventSum += lep.p4()
            #format_spec ::=  [[fill]align][sign][#][0][width][,][.precision][type]
            print("{0:*<5d} {1:>10.4f} {2:>+10.3f} {3:>+10.3f}".format(nm+1, lep.pt, lep.eta, lep.phi))
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Electron", "Pt", "Eta", "Phi"))
        for ne, lep in enumerate(electrons) :
            eventSum += lep.p4()
            print("{0:*^5d} {1:>10.4f} {2:> 10.3f} {3:> 10.3f}".format(ne+1, lep.pt, lep.eta, lep.phi))
        #for j in filter(self.jetSel,jets):
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Jet", "Pt", "Eta", "Phi"))
        for nj, j in enumerate(jets):
            eventSum += j.p4()
            print("{0: >5d} {1:>10.4f} {2:>-10.3f} {3:>-10.3f}".format(nj+1, j.pt, j.eta, j.phi))
        #for nt, trig in enumerate(triggers):
        #    if(nt < 5): print("TypeName: " + trig.GetTypeName())
        #idea: create list of names for triggers, then check bits with "triggers.name for name in names"
        print("HLT_PFMETNoMu90_PFMHTNoMu90_IDTight Trigger: " + triggers.PFMETNoMu90_PFMHTNoMu90_IDTight)
        print("Event Mass: {:<10.4f}\n".format(eventSum.M()))
        return True

# define modules using the syntax 'name = lambda : constructor' to avoid having them loaded when not needed

# testing = lambda : testing(jetSelection= lambda j : j.pt > 30)

In [156]:
#preselection="Jet_pt[0] > 250"
preselection="nMuon > 0 && nElectron > 0"
files=[" root://cms-xrd-global.cern.ch//store/mc/RunIISummer16NanoAOD/TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/NANOAODSIM/PUMoriond17_\
05Feb2018_94X_mcRun2_asymptotic_v2-v1/40000/2CE738F9-C212-E811-BD0E-EC0D9A8222CE.root"]
print(files)

[u' root://cms-xrd-global.cern.ch//store/mc/RunIISummer16NanoAOD/TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/NANOAODSIM/PUMoriond17_05Feb2018_94X_mcRun2_asymptotic_v2-v1/40000/2CE738F9-C212-E811-BD0E-EC0D9A8222CE.root']


In [157]:
#p=PostProcessor(".",files,cut=preselection,branchsel=None,modules=[testing()],noOut=True,histFileName="histOut.root",histDirName="plots")
p=PostProcessor(".",files,cut=preselection,branchsel=None,modules=[testing()],friend=False,postfix="AccessAndLoop", jsonInput=None,
                noOut=False,justcount=False,provenance=False,haddFileName=None,fwkJobReport=False,histFileName=None,
                histDirName=None, outputbranchsel=None)

In [154]:
p.run()

Will write selected trees to .
Pre-select 468497 entries out of 1386834 
Run:        1 Event:   145013
PV  X: 0.106 Y: 0.170 Z: -3.108
PV  X: 0.106 Y: 0.170 Z: -3.108 nDoF: 95.500000 Chi^2: 0.879
nSV:   1 SV[0] Decay Length:0.599
 Muon         Pt        Eta        Phi
1****    10.4301     +2.212     +1.385
2****     5.7181     +0.355     -0.389
Electron         Pt        Eta        Phi
**1**     5.0823      0.550      1.295
  Jet         Pt        Eta        Phi
    1    59.5000      2.309      1.351
    2    54.5312      0.693     -2.048
    3    44.0000      0.252     -0.333
    4    40.8125      2.755     -0.990
    5    38.9688      1.495      1.870
    6    37.8125      1.443      2.516
    7    35.1562      1.525      2.925
    8    22.4688     -3.195      1.143
    9    17.8438      4.054     -0.340
False


TypeError: 'int' object has no attribute '__getitem__'